In [1]:
import torch
import torch.nn as nn
import torch.nn.functional
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv("labeled_file.txt", sep="\t", header=None)
test_data = pd.read_csv("unlabeled_file.txt", sep="\t", header=None)

In [3]:
layers_dim = [21, 40, 60, 128]

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = torch.nn.Linear(layers_dim[0], layers_dim[1])
        self.tanh = torch.nn.Tanh()
        self.fc2 = torch.nn.Linear(layers_dim[1], layers_dim[2])
        self.relu = torch.nn.ReLU()
        self.drop = torch.nn.Dropout(0.5)
        self.fc3 = torch.nn.Linear(layers_dim[2], layers_dim[3])
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc3(x)
        
        return x
    
net = Net()

In [5]:
#USE THIS ONLY FOR LOADING THE MODEL

netLOADER = torch.nn.Sequential(
    torch.nn.Linear(layers_dim[0], layers_dim[1]),
    torch.nn.Tanh(), 
    torch.nn.Linear(layers_dim[1], layers_dim[2]),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(layers_dim[2], layers_dim[3]),
)

In [6]:
train_x = train_data.iloc[:, 2:-1]
test_x = test_data.iloc[:, 2:-1]
train_y = train_data.iloc[:, -1]
train_y.value_counts()

1      1382
0       462
7       182
12      170
2       146
5       125
11      101
15       99
10       98
18       83
9        81
8        80
30       78
13       71
25       63
27       61
3        61
4        58
49       56
16       48
34       45
38       45
45       43
19       41
29       37
41       32
39       31
24       29
53       28
64       27
       ... 
115       3
54        3
123       3
51        3
93        3
113       3
127       2
108       2
118       2
111       2
95        2
91        2
79        2
55        2
47        2
35        2
126       2
122       2
98        2
112       2
94        2
86        2
66        2
117       2
109       2
81        2
77        2
120       2
116       2
125       2
Name: 23, Length: 128, dtype: int64

In [7]:
print(train_x.head())
print(test_x.head())
print(train_y.head())

     2     3     4     5     6     7     8     9     10    11  ...     13  \
0  17.3   9.3  27.3  22.5  30.6  16.3  29.7  33.6  36.5  28.8  ...   33.6   
1  28.0  34.4  13.2  28.3  15.7  56.5  35.2  25.0  17.1  40.8  ...   20.4   
2  42.3  40.1   6.9  14.1  10.6   3.0  26.7  24.3  23.9   4.1  ...   11.1   
3  31.0   8.6  24.3   3.4  33.7  22.3   8.9  23.5  45.6  27.9  ...   31.4   
4  31.7  23.9  25.5  19.9  39.4  21.6  17.9  19.5  49.0  15.5  ...   56.0   

     14    15    16    17    18    19    20    21    22  
0  15.3  34.0  20.9  23.9  54.1  54.3  10.7  11.8  10.4  
1  11.6  15.9   5.9  18.2  28.8  55.0  35.9  25.2  18.8  
2   5.3  21.1  26.6  28.3  16.6  43.4  20.2  13.9  11.8  
3  26.5  27.7  26.9   8.2  58.3  58.6  39.5  16.9  36.4  
4  18.8  27.0  15.5  19.2  44.6  35.9  54.8  22.9  27.0  

[5 rows x 21 columns]
       2       3       4       5       6       7       8       9       10  \
0   295.5   392.5   265.5   338.5   461.9   225.3   671.4   364.6   237.0   
1  2359.3  1

In [8]:
print(net.parameters)

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=21, out_features=40, bias=True)
  (tanh): Tanh()
  (fc2): Linear(in_features=40, out_features=60, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.5)
  (fc3): Linear(in_features=60, out_features=128, bias=True)
)>


In [9]:
train_x = np.array(train_x.values, dtype=np.float32)
train_y = np.array(train_y.values, dtype=np.long)
test_x = np.array(test_x.values, dtype=np.float32)
train_y = np.reshape(train_y, (4636, ))
train_x = torch.from_numpy(train_x)
test_x = torch.from_numpy(test_x)
train_y = torch.from_numpy(train_y)

In [10]:
print(train_x.shape, test_x.shape, train_y.shape)

torch.Size([4636, 21]) torch.Size([19, 21]) torch.Size([4636])


In [11]:
def predict_accuracy(mynet, val_x, val_y):
    outputs = mynet(val_x)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == val_y).sum().item()
    print("Accuracy = {}%".format(100 *(float(correct/val_x.shape[0]))))

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
num_epochs = 10000

In [13]:
for epoch in range(num_epochs):  # number of epochs
    outputs = net(train_x)
    loss = criterion(outputs, train_y)
    
    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    predict_accuracy(net, train_x, train_y)

Epoch [1/10000], Loss: 4.9226
Accuracy = 0.7765314926660914%
Epoch [2/10000], Loss: 4.8823
Accuracy = 0.992234685073339%
Epoch [3/10000], Loss: 4.8395
Accuracy = 0.819672131147541%
Epoch [4/10000], Loss: 4.7961
Accuracy = 1.5099223468507335%
Epoch [5/10000], Loss: 4.7600
Accuracy = 1.2510785159620361%
Epoch [6/10000], Loss: 4.7157
Accuracy = 2.5237273511647973%
Epoch [7/10000], Loss: 4.6751
Accuracy = 3.6669542709232097%
Epoch [8/10000], Loss: 4.6346
Accuracy = 4.918032786885246%
Epoch [9/10000], Loss: 4.5898
Accuracy = 6.492666091458153%
Epoch [10/10000], Loss: 4.5472
Accuracy = 9.10267471958585%
Epoch [11/10000], Loss: 4.5007
Accuracy = 10.547886108714408%
Epoch [12/10000], Loss: 4.4598
Accuracy = 12.683347713546162%
Epoch [13/10000], Loss: 4.4033
Accuracy = 13.481449525452977%
Epoch [14/10000], Loss: 4.3644
Accuracy = 16.371872303710095%
Epoch [15/10000], Loss: 4.3126
Accuracy = 16.45815358067299%
Epoch [16/10000], Loss: 4.2610
Accuracy = 19.84469370146678%
Epoch [17/10000], Loss: 4

Epoch [135/10000], Loss: 3.0439
Accuracy = 30.737704918032787%
Epoch [136/10000], Loss: 3.0423
Accuracy = 31.082830025884384%
Epoch [137/10000], Loss: 3.0425
Accuracy = 30.910267471958587%
Epoch [138/10000], Loss: 3.0388
Accuracy = 30.69456427955134%
Epoch [139/10000], Loss: 3.0411
Accuracy = 30.414150129421913%
Epoch [140/10000], Loss: 3.0380
Accuracy = 30.629853321829163%
Epoch [141/10000], Loss: 3.0314
Accuracy = 30.58671268334771%
Epoch [142/10000], Loss: 3.0237
Accuracy = 30.996548748921484%
Epoch [143/10000], Loss: 3.0262
Accuracy = 31.061259706643657%
Epoch [144/10000], Loss: 3.0261
Accuracy = 31.12597066436583%
Epoch [145/10000], Loss: 3.0212
Accuracy = 30.457290767903366%
Epoch [146/10000], Loss: 3.0239
Accuracy = 30.737704918032787%
Epoch [147/10000], Loss: 3.0190
Accuracy = 30.780845556514237%
Epoch [148/10000], Loss: 3.0199
Accuracy = 30.75927523727351%
Epoch [149/10000], Loss: 3.0139
Accuracy = 30.910267471958587%
Epoch [150/10000], Loss: 3.0159
Accuracy = 31.0828300258843

Epoch [267/10000], Loss: 2.8890
Accuracy = 31.837791199309752%
Epoch [268/10000], Loss: 2.8787
Accuracy = 31.85936151855047%
Epoch [269/10000], Loss: 2.8825
Accuracy = 32.26919758412424%
Epoch [270/10000], Loss: 2.8878
Accuracy = 32.01035375323555%
Epoch [271/10000], Loss: 2.8822
Accuracy = 32.78688524590164%
Epoch [272/10000], Loss: 2.8790
Accuracy = 32.09663503019844%
Epoch [273/10000], Loss: 2.8850
Accuracy = 32.70060396893874%
Epoch [274/10000], Loss: 2.8780
Accuracy = 32.35547886108714%
Epoch [275/10000], Loss: 2.8770
Accuracy = 32.42018981880932%
Epoch [276/10000], Loss: 2.8731
Accuracy = 31.945642795513372%
Epoch [277/10000], Loss: 2.8733
Accuracy = 32.4849007765315%
Epoch [278/10000], Loss: 2.8713
Accuracy = 32.44176013805004%
Epoch [279/10000], Loss: 2.8719
Accuracy = 32.57118205349439%
Epoch [280/10000], Loss: 2.8659
Accuracy = 31.92407247627265%
Epoch [281/10000], Loss: 2.8679
Accuracy = 32.139775668679896%
Epoch [282/10000], Loss: 2.8676
Accuracy = 32.33390854184642%
Epoch 

Accuracy = 33.584987057808455%
Epoch [402/10000], Loss: 2.7584
Accuracy = 34.21052631578947%
Epoch [403/10000], Loss: 2.7652
Accuracy = 33.8869715271786%
Epoch [404/10000], Loss: 2.7538
Accuracy = 33.73597929249353%
Epoch [405/10000], Loss: 2.7546
Accuracy = 34.29680759275237%
Epoch [406/10000], Loss: 2.7535
Accuracy = 34.21052631578947%
Epoch [407/10000], Loss: 2.7457
Accuracy = 34.1458153580673%
Epoch [408/10000], Loss: 2.7514
Accuracy = 34.01639344262295%
Epoch [409/10000], Loss: 2.7415
Accuracy = 33.9732528041415%
Epoch [410/10000], Loss: 2.7456
Accuracy = 33.908541846419325%
Epoch [411/10000], Loss: 2.7427
Accuracy = 34.081104400345126%
Epoch [412/10000], Loss: 2.7355
Accuracy = 34.68507333908542%
Epoch [413/10000], Loss: 2.7406
Accuracy = 33.671268334771355%
Epoch [414/10000], Loss: 2.7302
Accuracy = 33.95168248490077%
Epoch [415/10000], Loss: 2.7438
Accuracy = 33.7144089732528%
Epoch [416/10000], Loss: 2.7351
Accuracy = 34.21052631578947%
Epoch [417/10000], Loss: 2.7279
Accuracy

Epoch [534/10000], Loss: 2.6348
Accuracy = 35.526315789473685%
Epoch [535/10000], Loss: 2.6405
Accuracy = 36.022433132010356%
Epoch [536/10000], Loss: 2.6402
Accuracy = 35.74201898188093%
Epoch [537/10000], Loss: 2.6384
Accuracy = 36.324417601380496%
Epoch [538/10000], Loss: 2.6378
Accuracy = 36.21656600517687%
Epoch [539/10000], Loss: 2.6436
Accuracy = 36.28127696289905%
Epoch [540/10000], Loss: 2.6431
Accuracy = 35.82830025884383%
Epoch [541/10000], Loss: 2.6377
Accuracy = 35.95772217428818%
Epoch [542/10000], Loss: 2.6365
Accuracy = 36.087144089732526%
Epoch [543/10000], Loss: 2.6293
Accuracy = 35.56945642795513%
Epoch [544/10000], Loss: 2.6239
Accuracy = 36.43226919758412%
Epoch [545/10000], Loss: 2.6376
Accuracy = 36.022433132010356%
Epoch [546/10000], Loss: 2.6272
Accuracy = 36.62640207075064%
Epoch [547/10000], Loss: 2.6249
Accuracy = 36.798964624676444%
Epoch [548/10000], Loss: 2.6262
Accuracy = 36.259706643658326%
Epoch [549/10000], Loss: 2.6259
Accuracy = 36.259706643658326%


...

Epoch [9431/10000], Loss: 5.5285
Accuracy = 33.110440034512514%
Epoch [9432/10000], Loss: 5.4047
Accuracy = 35.69887834339948%
Epoch [9433/10000], Loss: 5.1607
Accuracy = 36.345987920621226%
Epoch [9434/10000], Loss: 4.7599
Accuracy = 35.72044866264021%
Epoch [9435/10000], Loss: 4.1439
Accuracy = 35.181190681622084%
Epoch [9436/10000], Loss: 3.8071
Accuracy = 34.53408110440034%
Epoch [9437/10000], Loss: 3.5266
Accuracy = 32.74374460742019%
Epoch [9438/10000], Loss: 3.3365
Accuracy = 32.33390854184642%
Epoch [9439/10000], Loss: 3.2032
Accuracy = 32.52804141501294%
Epoch [9440/10000], Loss: 3.1858
Accuracy = 32.614322691975836%
Epoch [9441/10000], Loss: 3.1074
Accuracy = 33.69283865401208%
Epoch [9442/10000], Loss: 2.8837
Accuracy = 35.46160483175151%
Epoch [9443/10000], Loss: 2.7515
Accuracy = 36.022433132010356%
Epoch [9444/10000], Loss: 2.6266
Accuracy = 35.13805004314064%
Epoch [9445/10000], Loss: 2.6209
Accuracy = 33.82226056945643%
Epoch [9446/10000], Loss: 2.6256
Accuracy = 33.132

Epoch [9563/10000], Loss: 1.7117
Accuracy = 52.39430543572045%
Epoch [9564/10000], Loss: 1.7045
Accuracy = 51.87661777394305%
Epoch [9565/10000], Loss: 1.7068
Accuracy = 52.07075064710958%
Epoch [9566/10000], Loss: 1.6949
Accuracy = 52.221742881794654%
Epoch [9567/10000], Loss: 1.7053
Accuracy = 52.717860224331325%
Epoch [9568/10000], Loss: 1.7018
Accuracy = 52.58843830888698%
Epoch [9569/10000], Loss: 1.7007
Accuracy = 52.459016393442624%
Epoch [9570/10000], Loss: 1.6843
Accuracy = 52.15703192407247%
Epoch [9571/10000], Loss: 1.6982
Accuracy = 52.480586712683355%
Epoch [9572/10000], Loss: 1.6901
Accuracy = 53.25711820534944%
Epoch [9573/10000], Loss: 1.6833
Accuracy = 52.782571182053495%
Epoch [9574/10000], Loss: 1.6936
Accuracy = 52.610008628127694%
Epoch [9575/10000], Loss: 1.6778
Accuracy = 52.4374460742019%
Epoch [9576/10000], Loss: 1.6876
Accuracy = 53.77480586712683%
Epoch [9577/10000], Loss: 1.6855
Accuracy = 53.343399482312336%
Epoch [9578/10000], Loss: 1.6884
Accuracy = 52.67

Epoch [9693/10000], Loss: 1.5585
Accuracy = 55.88869715271786%
Epoch [9694/10000], Loss: 1.5692
Accuracy = 54.7886108714409%
Epoch [9695/10000], Loss: 1.5607
Accuracy = 55.69456427955134%
Epoch [9696/10000], Loss: 1.5642
Accuracy = 55.65142364106988%
Epoch [9697/10000], Loss: 1.5565
Accuracy = 55.88869715271786%
Epoch [9698/10000], Loss: 1.5656
Accuracy = 55.996548748921484%
Epoch [9699/10000], Loss: 1.5592
Accuracy = 55.67299396031061%
Epoch [9700/10000], Loss: 1.5568
Accuracy = 56.38481449525453%
Epoch [9701/10000], Loss: 1.5621
Accuracy = 55.457290767903366%
Epoch [9702/10000], Loss: 1.5559
Accuracy = 55.78084555651424%
Epoch [9703/10000], Loss: 1.5637
Accuracy = 55.67299396031061%
Epoch [9704/10000], Loss: 1.5517
Accuracy = 55.15530629853321%
Epoch [9705/10000], Loss: 1.5576
Accuracy = 55.95340811044004%
Epoch [9706/10000], Loss: 1.5539
Accuracy = 56.03968938740294%
Epoch [9707/10000], Loss: 1.5539
Accuracy = 55.80241587575496%
Epoch [9708/10000], Loss: 1.5494
Accuracy = 55.6514236

Accuracy = 51.05694564279551%
Epoch [9824/10000], Loss: 1.6464
Accuracy = 48.20966350301985%
Epoch [9825/10000], Loss: 1.7657
Accuracy = 48.61949956859362%
Epoch [9826/10000], Loss: 1.7390
Accuracy = 51.29421915444349%
Epoch [9827/10000], Loss: 1.6116
Accuracy = 55.457290767903366%
Epoch [9828/10000], Loss: 1.5198
Accuracy = 52.97670405522001%
Epoch [9829/10000], Loss: 1.6033
Accuracy = 56.57894736842105%
Epoch [9830/10000], Loss: 1.5166
Accuracy = 51.68248490077653%
Epoch [9831/10000], Loss: 1.6337
Accuracy = 48.14495254529767%
Epoch [9832/10000], Loss: 1.7546
Accuracy = 48.08024158757549%
Epoch [9833/10000], Loss: 1.7527
Accuracy = 54.44348576358931%
Epoch [9834/10000], Loss: 1.5567
Accuracy = 52.480586712683355%
Epoch [9835/10000], Loss: 1.6065
Accuracy = 45.599654874892146%
Epoch [9836/10000], Loss: 1.8254
Accuracy = 46.78602243313201%
Epoch [9837/10000], Loss: 1.8022
Accuracy = 53.32182916307161%
Epoch [9838/10000], Loss: 1.5610
Accuracy = 47.26056945642795%
Epoch [9839/10000], Lo

Epoch [9953/10000], Loss: 1.5315
Accuracy = 57.16134598792062%
Epoch [9954/10000], Loss: 1.5288
Accuracy = 56.92407247627265%
Epoch [9955/10000], Loss: 1.5353
Accuracy = 57.09663503019845%
Epoch [9956/10000], Loss: 1.5343
Accuracy = 56.68679896462467%
Epoch [9957/10000], Loss: 1.5293
Accuracy = 56.514236410698885%
Epoch [9958/10000], Loss: 1.5275
Accuracy = 57.20448662640207%
Epoch [9959/10000], Loss: 1.5348
Accuracy = 56.57894736842105%
Epoch [9960/10000], Loss: 1.5310
Accuracy = 57.139775668679896%
Epoch [9961/10000], Loss: 1.5249
Accuracy = 56.988783433994826%
Epoch [9962/10000], Loss: 1.5331
Accuracy = 56.92407247627265%
Epoch [9963/10000], Loss: 1.5286
Accuracy = 56.8809318377912%
Epoch [9964/10000], Loss: 1.5262
Accuracy = 57.16134598792062%
Epoch [9965/10000], Loss: 1.5198
Accuracy = 56.729939603106125%
Epoch [9966/10000], Loss: 1.5261
Accuracy = 57.2260569456428%
Epoch [9967/10000], Loss: 1.5253
Accuracy = 57.355478861087136%
Epoch [9968/10000], Loss: 1.5271
Accuracy = 57.57118

In [14]:
#outputs = net(train_x)

In [15]:
#_, predicted = torch.max(outputs, 1)

In [ ]:
#print(predicted)

In [ ]:
#correct = (predicted == train_y).sum().item()

In [ ]:
#print("Accuracy = {}%".format(100 *(float(correct/train_data.shape[0]))))

In [ ]:
predict_accuracy(net, train_x, train_y)

In [ ]:
def generate_labels(test_x):
    outputs = net(test_x)
    _, predictions = torch.max(outputs, 1)
    return predictions

In [ ]:
torch.save(net, "trained_model_10000_epochs_4.pt")

In [ ]:
netLOADER = torch.load("trained_model_10000_epochs_3.pt")

In [ ]:
print(netLOADER)

In [ ]:
test_labels = generate_labels(test_x)
print(test_labels)

In [ ]:
test_data[[23]] = test_labels
test_data

In [ ]:
test_data.to_csv("Labelled_prev_unlabelBCLL_NN_2hiddenL.txt", sep=" ", header=False, index=False)